In [ ]:
# 기본 라이브러리 호출
import warnings
warnings.filterwarnings('ignore')
warnings.warn("once")
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew
# 선형모델을 추정하는 라이브러리
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.stats.api as sms
from patsy import dmatrices
color = sns.color_palette()

pd.set_option('display.float_format','{:,.4f}'.format) # 소수점 2번째 자리까지 표현
pd.set_option('display.max_columns', None) # 모든 컬럼 표시
pd.set_option('display.max_colwidth', -1) # 컬럼내용 전체 표시

#Graph에 한글을 표시하기 위한 코드
import matplotlib
from matplotlib import font_manager, rc
import platform
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
font_name = font_manager.FontProperties(fname="/home/spa/.local/share/Trash/files/one/malgun.ttf").get_name()
rc('font', family=font_name)\
    
matplotlib.rcParams['axes.unicode_minus'] = False

In [ ]:
pwd

# 1. 정형 데이터마이닝 (사용 데이터 : lotto)

### Q1) 연관규칙분석을 수행하기 위해 lotto 데이터셋을 transaction 데이터로 변환하시오. 
###     단, 본 분석에서 로또번호가 추첨된 순서는 고려하지 않고 분석을 수행하도록 한다. 
###     그리고 변환된 데이터에서 가장 많이 등장한 
###     상위 10개의 로또번호를 막대그래프로 출력하고 이에 대해 설명하시오.

In [ ]:
df = pd.read_csv('./data/lotto.csv')
print(df.shape)
df.head()

In [ ]:
melt = pd.melt(df, id_vars = ['time_id'])

In [ ]:
melt

In [ ]:
pivot = melt.groupby('value')[['variable']].count().sort_values(by = 'variable', ascending = False)[:10].reset_index()

In [ ]:
pivot

In [ ]:
sns.barplot(x = 'value', y = 'variable', data = pivot, order = pivot['value'])

### Q2) 변환한 데이터에 대해 apriori함수를 사용하여 다음 괄호 안의 조건을 반영하여 
###     연관규칙을 생성하고, 이를 ‘rules_1’이라는 변수에 저장하여 결과를 해석하시오. 
###     (최소 지지도 : 0.002, 최소 신뢰도 : 0.8, 최소조합 항목 수 : 2개, 최대조합 항목 수 : 6개) 
###     그리고 도출된 연관규칙들을 향상도를 기준으로 내림차순 정렬하여 상위 30개의 규칙을 확인하고, 
###     이를 데이터프레임으로 변환하여 csv파일로 출력하시오.  

In [ ]:
melt

In [ ]:
df.head()

In [ ]:
df.columns[1:7]

In [ ]:
result = []
column_list = df.columns[1:7]
for i in range(len(df)):
    num_list = []
    for j in column_list:
        num_list.append(df[j][i])
    result.append(num_list)

In [ ]:
num_list

In [ ]:
result[:10]

In [ ]:
# !pip install mlxtend

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [ ]:
te = TransactionEncoder()
te_result = te.fit(result).transform(result)

In [ ]:
te_result

In [ ]:
df = pd.DataFrame(te_result, columns=te.columns_)

In [ ]:
df

In [ ]:
frequent_itemsets = apriori(df, min_support=0.002, use_colnames=True)
frequent_itemsets

In [ ]:
from mlxtend.frequent_patterns import association_rules
result_df = association_rules(frequent_itemsets, metric="lift", min_threshold=0.8)

In [ ]:
result_df.sort_values(by = 'support', ascending = False)[:10]

In [ ]:
result_df.sort_values(by = 'lift', ascending = False)[:10]

In [ ]:
rules_1 = result_df[result_df['antecedents'].apply(lambda x: len(x) >= 2)]

In [ ]:
rules_1 = rules_1[rules_1['confidence'] >= 0.8]

In [ ]:
rules_1.sort_values(by = 'lift', ascending = False)[:10]

In [ ]:
rules_1[rules_1['consequents'] == frozenset({34})].sort_values(by = 'lift', ascending = False)[:10]

# 2. 통계분석 (사용 데이터 : FIFA)

### Q1) FIFA데이터에서 각 선수의 키는 Heghit변수에 피트와 인치로 입력되어 있습니다. 
###      이를 cm로 변환하여 새로운 변수 Height_cm을 생성하시오. 
###      ( “ ' ” 앞의 숫자는 피트이며, “ ' ” 뒤의 숫자는 인치, 1피트 = 30cm, 1인치 = 2.5cm)

In [ ]:
df = pd.read_csv('./data/FIFA.csv', encoding = 'cp949')
print(df.shape)
df.head()

In [ ]:
df['Height']

In [ ]:
temp_list = df['Height'].str.split("'")

In [ ]:
cm = []
for i in temp_list:
    cm.append(int(i[0])* 30 + (int(i[1])* 2.5))

In [ ]:
df['Height_cm'] = cm

In [ ]:
df['Height_cm'] = df['Height_cm'].round(0)

In [ ]:
df[['Height', 'Height_cm']]

### Q2) 포지션을 의미하는 Position변수를 아래 표를 참고하여 “Forward”, “Midfielder”, 
###      “Defender”, “GoalKeeper”로 재범주화하고, factor형으로 변환하여 Position_Class 
###       라는 변수를 생성하고 저장하시오.

In [ ]:
df['Position'].unique()

In [ ]:
foward = ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW']
mid = ['LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM']
defender = ['LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB']
gk = ['GK']

In [ ]:
df.loc[df['Position'].isin(foward), 'Position_Class'] = 'Forward'
df.loc[df['Position'].isin(mid), 'Position_Class'] = 'Midfielder'
df.loc[df['Position'].isin(defender), 'Position_Class'] = 'Defender'
df.loc[df['Position'].isin(gk), 'Position_Class'] = 'GoalKeeper'

In [ ]:
df['Position_Class'].unique()

### Q3) 새로 생성한 Position_Class 변수의 각 범주에 따른 Value(선수의 시장가치)의 
###      평균값의 차이를 비교하는 일원배치 분산분석을 수행하고 결과를 해석하시오. 
###      그리고 평균값의 차이가 통계적으로 유의하다면 사후검정을 수행하고 설명하시오.

In [ ]:
df.groupby(['Position_Class'])[['Value']].mean()

### Anova 가정

* 독립변인/독립변수의 그룹 군은 서로 독립
* 독립변인에 대한 종속변인은 정규분포
* 독립변인에 따른 종속변인의 분산은 등분산

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('Value ~ C(Position_Class)', df).fit()
anova_lm(model)

* 분산분석표를 통해 확인한 결과, SSA의 자유도는 3(집단의 수 -1 = 4-1), SST의 자유도는 16638(관측값의 수 - 집단의 수 = 16642-4)임을 확인. p값은 매우 작아 유의수준 0.05하에서 귀무가설을 기각한다. 따라서 포지션별 시장가치가 모두 동일하지 않다고 결론. 포지션별 시장가치의 평균값들 중 적어도 어느 하나의 포지션은 통계적으로 유의한 차이가 있다고 말할 수 있다.

### 사후 검정

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

posthoc = pairwise_tukeyhsd(df['Value'], df['Position_Class'], alpha=0.05)
print(posthoc)

* 사후검정에서는 귀무가설을 '집단들 사이의 평균은 같다', 대립가설을 ' 집단들 사이의 평균은 같지않다'로 두고, 모든 집단 수준에 대해서 두 집단씩 짝을 지어 각각 다중 비교를 수행한다. 사후검정 결과를 보면 Midfielder~Forward 간의 비교에 대해 p값이 0.05보다 크므루 귀무가설을 기각하지 않는다. 즉, Midfielder와 Forward간의 시장가치는 통계적으로 유의한 차이가 있다고 볼수 없다. 그러나 다른 모든 포지션 간의 비교에 대해서는 p값이 0.05보다 작으므로 각가의 비교에 대한 귀무가설을 모두 기각한다. 즉 해당 포지션에 대한 시장가치 평균값은 각각 통계적으로 유의한 차이가 있다. 또한 meandiff는 왼쪽 집단과 오른쪽 집단간의 반응값의 차이를 나타내며, 양수인 경우 오른쪽의 평균이 더 높다.

### Q4) Preferred Foot(주로 사용하는 발)과 Position_Class(재범주화 된 포지션)변수에 
###      따라 Value(이적료)의 차이가 있는지를 알아보기 위해 이원배치분산분석을 
###      수행하고 결과를 해석하시오.

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('Value ~ C(Preferred_Foot) * C(Position_Class)', df).fit()
anova_lm(model)

* 주발의 p값이 0.05보다 작으므로 유의미한 차이. 주발에 따른 선수가치에 차이가 있다.
* 포지션의 p값이 0.05보다 작으므로 유의미한 차이. 포지션에 따른 선수가치에 차이가 있다.
* 주발과 포지션의 교호작용에 대한 p값은 0.05보다 작으므로 주발과 포지션의 상호작용에 의한 효과가 있음.

In [ ]:
df.head(3)

### Q5)Age, Overall, Wage, Height_cm, Weight_lb 가 Value에 영향을 미치는지 
###     알아보는 회귀분석을 단계적 선택법을 사용하여 수행하고 결과를 해석하시오.

In [ ]:
model = sm.OLS.from_formula('Value ~ Age + Overall + Wage + Height_cm + Weight_lb', data = df).fit()
model.summary()

In [ ]:
model = sm.OLS.from_formula('Value ~ Age + Overall + Wage + Height_cm', data = df).fit()
model.summary()

# 3. 비정형 데이터마이닝 (사용 데이터 : "영화 review")

### Q1) ‘영화 기생충_review.txt’ 데이터를 읽어온 뒤 숫자, 특수 문자 등을 
###     제거하는 전처리 작업을 시행하시오. 그리고 ‘영화 기생충_review.txt’을 사전에 등록하라.

In [ ]:
file = open('./data/영화 기생충_review.txt', 'r', encoding='cp949')
line = file.read()

In [ ]:
line = line.split('\n')

In [ ]:
text = pd.Series(line)

In [ ]:
# !pip install konlpy

In [ ]:
import konlpy
import re

In [ ]:
import re

text = text.map(lambda x: re.sub(r"\d", " ", x))
text = text.map(lambda x: re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》;]', ' ', x))
text

In [ ]:
dicts = pd.read_csv('./data/모의고사 1회/영화 기생충_사전.txt', sep = '\t', encoding = 'cp949', names = ['word'])

In [ ]:
dicts

In [ ]:
dicts['pos'] = 'NNP'

In [ ]:
dicts.to_csv('./bugs.txt', sep='\t', index=False, header=False)

In [ ]:
from konlpy.tag import Komoran
komoran = Komoran(userdic='bugs.txt')

### Q2) ‘영화 기생충_사전.txt’를 단어 사전으로 하는 TDM을 구축하고 빈도를 파악하고 시각화하라.

In [ ]:
# 벡터화 인스턴스 생성
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

# 사전으로 fit
cv.fit(dicts['word'])

In [ ]:
# 문서로 tdm 생성
cv_matrix = cv.transform(text)

In [ ]:
name = cv.get_feature_names()
cv_mat = cv_matrix.toarray()
tdm = pd.DataFrame(cv_mat, columns = name)

In [ ]:
tdm

In [ ]:
# 빈도 계산
tdm_freq = tdm.sum(axis=0).sort_values(ascending=False)

In [ ]:
tdm_freq

In [ ]:
# 시각화
tdm_freq.plot(kind = 'bar', figsize=(10,5))
plt.show()

In [ ]:
nouns = komoran.nouns(' '.join(text))
from collections import Counter
cnt = Counter(nouns)
temp_df = pd.DataFrame(columns = ['명사', '빈도'])
freq = cnt.most_common(10)
temp_list = []
temp_list2= []
for i in freq:
    temp_list.append(i[0])
    temp_list2.append(i[1])
temp_df['명사'] = temp_list
temp_df['빈도'] = temp_list2
temp_df

In [ ]:
words = {}
for i in cnt.most_common(20):
    words[i[0]] = i[1]

In [ ]:
# !pip install wordcloud

In [ ]:
from wordcloud import WordCloud 
import nltk
from nltk.corpus import stopwords

wordcloud = WordCloud(font_path = "/usr/share/fonts/NanumGothicCoding.ttf", background_color='white',colormap = "Accent_r",
                      width=1500, height=1000).generate_from_frequencies(words)

plt.imshow(wordcloud)
plt.axis('off')
plt.show()